# 最小二乗問題
### 問題の準備

In [1]:
using CSV
using DataFrames
using LinearAlgebra
using Random
using SparseArrays
using Plots

include("Algorithm.jl")
include("Operator.jl")

loss(x) = norm(A*x - b)^2/2
reg(x) = theta * sum(abs.(x))
obj(x) = loss(x) + reg(x)

grad(x) = A' * (A *x - b)

rng = MersenneTwister(1234)
n = 200
m = 300 # 自由に変えてもよいですが，n < m を推奨．
sparsity = 0.05
theta = .00015 # θ とすることも可能

A = randn(rng, Float64, (m, n))
foreach(normalize!, eachcol(A))
x = Array(sprandn(rng, n, sparsity))
b = A * x

lambda = 1.0

rng = MersenneTwister(5)
x0 = randn(rng, Float64, n)
maximum_iter = 10
tol = 1e-1

result_dir = "./results/least-squares_$n-$m-$sparsity-$theta"
if !ispath("./results")
    mkdir("./results")
end

### 近接勾配法を実行する
初回のコンパイルでは "94.45% compilation time"等と表示されます．
何回か繰り返すと表示されなくなるので，そのデータを使うと計算時間が比較できる．
これは Just In Time (JIT) コンパイルによる高速化が原因です（なので Julia は速い）．

In [2]:
pg = Algorithm.IterativeAlgorithm(x0, maximum_iter, tol)
@time Algorithm.run_ISTA(pg, obj, grad, lambda, theta)
CSV.write(result_dir * "-pg.csv", pg.df)

  0.147143 seconds (728.69 k allocations: 50.089 MiB, 6.38% gc time, 99.59% compilation time)


"./results/least-squares_200-300-0.05-0.00015-pg.csv"

In [3]:
# アルゴリズムを加える場合は Algorithm.jl に run_hoge を書き直しましょう．"hoge" や "HOGE" は適宜変更してください．
hoge = Algorithm.IterativeAlgorithm(x0, maximum_iter, tol)
@time Algorithm.run_hoge(hoge, obj, grad, lambda, theta)
CSV.write(result_dir * "-hoge.csv", hoge.df)

  0.009646 seconds (16.84 k allocations: 1.317 MiB, 95.76% compilation time)


"./results/least-squares_200-300-0.05-0.00015-hoge.csv"

### 横軸を反復数にしたもの

In [4]:
df_pg = pg.df
df_hoge = hoge.df
obj_opt = obj(x)
plot(df_pg.iter, abs.(df_pg.obj .- obj_opt), xaxis="k", yaxis=raw"$\log_{10}|Ψ(x^k) - Ψ(x^*)|$", label="PG", yscale=:log10)
plot!(df_hoge.iter, abs.(df_hoge.obj .- obj_opt), xaxis="k", yaxis=raw"$\log_{10}|Ψ(x^k) - Ψ(x^*)|$", label="HOGE", yscale=:log10)
savefig(result_dir * "-diffobj.png")
print("saved ", result_dir * "-diffobj.png\n")
plot(df_pg.iter, df_pg.obj, xaxis="k", yaxis=raw"$\log_{10}|Ψ(x^k)|$", label="PG", yscale=:log10)
plot!(df_hoge.iter, df_hoge.obj, xaxis="k", yaxis=raw"$\log_{10}|Ψ(x^k)|$", label="HOGE", yscale=:log10)
savefig(result_dir * "-objv.png")
print("saved ", result_dir * "-objv.png\n")

saved ./results/least-squares_200-300-0.05-0.00015-diffobj.png
saved ./results/least-squares_200-300-0.05-0.00015-objv.png


### 横軸を時間にしたもの
提出は反復数を横軸にしたグラフのみで十分ですが（もちろん，提出しても構いません），計算時間を横軸にして比較も可能です．
しかし，上で述べた通り，初回実行時はコンパイル処理が含まれてしまうので，何回かアルゴリズムを実行して，コンパイルが含まれていない結果をプロットすると公平に比較ができます．

In [5]:
plot(df_pg.time, abs.(df_pg.obj .- obj_opt), xaxis="time (sec)", yaxis=raw"$\log_{10}|Ψ(x^k) - Ψ(x^*)|$", label="PG", yscale=:log10)
plot!(df_hoge.time, abs.(df_hoge.obj .- obj_opt), xaxis="time (sec)", yaxis=raw"$\log_{10}|Ψ(x^k) - Ψ(x^*)|$", label="HOGE", yscale=:log10)
savefig(result_dir * "-diffobj-time.png")
print("saved ", result_dir * "-diffobj-time.png\n")
plot(df_pg.time, df_pg.obj, xaxis="time (sec)", yaxis=raw"$\log_{10}|Ψ(x^k)|$", label="PG", yscale=:log10)
plot!(df_hoge.time, df_hoge.obj, xaxis="time (sec)", yaxis=raw"$\log_{10}|Ψ(x^k)|$", label="HOGE", yscale=:log10)
savefig(result_dir * "-objv-time.png")
print("saved ", result_dir * "-objv-time.png\n")

saved ./results/least-squares_200-300-0.05-0.00015-diffobj-time.png
saved ./results/least-squares_200-300-0.05-0.00015-objv-time.png
